In [1]:
import postgresql

In [2]:
db = postgresql.open('pq://derin:qwerty@localhost:5432/models')

In [6]:
db.query("SELECT column_name FROM model_1_level, morf_characters_of_word")
#Надо научиться писать select, чтобы красиво было

UndefinedColumnError: column "column_name" does not exist
  CODE: 42703
  LOCATION: File 'parse_relation.c', line 3194, in errorMissingColumn from SERVER
  POSITION: 8
STATEMENT: [parsing]
  LINE:
    SELECT column_name FROM model_1_level, morf_characters_of_word
           ^ [line 1, character 8]
  statement_id: py:0x7f21fc5a7668
  string: SELECT column_name FROM model_1_level, morf_characters_of_word
CONNECTION: [idle]
  client_address: 127.0.0.1/32
  client_port: 54276
  version:
    PostgreSQL 9.5.14 on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 5.4.0-6ubuntu1~16.04.10) 5.4.0 20160609, 64-bit
CONNECTOR: [Host] pq://derin:***@localhost:5432/models
  category: None
DRIVER: postgresql.driver.pq3.Driver

In [7]:
word = "смотреть"
нужны функции
1. на вход - слово, на выход все модели с ним и 

In [17]:
#по главному слову - все модели 3 уровня
word = "видеть"
s = "WITH w AS (SELECT number_word FROM word WHERE word_text = \'" + word +"\') SELECT ref_to_dep_word,imp_feat_main, imp_feat_dep, prep, mark FROM model_3_level, w WHERE model_3_level.ref_to_main_word = w.number_word;"
print(s)
res = db.query(s)

WITH w AS (SELECT number_word FROM word WHERE word_text = 'видеть') SELECT ref_to_dep_word,imp_feat_main, imp_feat_dep, prep, mark FROM model_3_level, w WHERE model_3_level.ref_to_main_word = w.number_word;


In [18]:
print("ref_to_dep_word,imp_feat_main, imp_feat_dep, prep, mark")
for i in res:
    print(i)

ref_to_dep_word,imp_feat_main, imp_feat_dep, prep, mark
(3, 1, 2, 122, 0.5)
(4, 1, 2, 122, 0.5)


In [19]:
res1 = db.query("WITH w AS (SELECT number_word FROM word WHERE word_text = 'видеть'), mod AS (SELECT ref_to_dep_word,imp_feat_main, imp_feat_dep, prep, mark FROM model_3_level, w WHERE model_3_level.ref_to_main_word = w.number_word), impMain AS (SELECT * FROM mod, important_features WHERE mod.imp_feat_main = important_features.number_imp_feat), impDep AS (SELECT * FROM mod, important_features WHERE mod.imp_feat_dep = important_features.number_imp_feat) SELECT * FROM impMain, impDep;")

In [20]:
res1

[(4,
  1,
  2,
  122,
  0.5,
  1,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  4,
  1,
  2,
  122,
  0.5,
  2,
  True,
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False),
 (4,
  1,
  2,
  122,
  0.5,
  1,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  3,
  1,
  2,
  122,
  0.5,
  2,
  True,
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False),
 (3,
  1,
  2,
  122,
  0.5,
  1,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  4,
  1,
  2,
  122,
  0.5,
  2,
  True,
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False),
 (3,
  1,
  2,
  122,
  0.5,
  1,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  True,

In [41]:
columns_imp_feat = db.query(" SELECT column_name FROM information_schema.columns WHERE table_name = 'important_features' AND table_schema = 'public';")

In [45]:
word = "видеть"
imp_dep = db.query("WITH w AS (SELECT number_word FROM word WHERE word_text = \'" + word + "\'), mod AS (SELECT ref_to_dep_word,imp_feat_main, imp_feat_dep, prep, mark FROM model_3_level, w WHERE model_3_level.ref_to_main_word = w.number_word), impDep AS (SELECT important_features.* FROM mod, important_features WHERE mod.imp_feat_dep = important_features.number_imp_feat) SELECT * FROM impDep;")

In [46]:
for str1 in imp_dep:
    for elem in str1:
        for name in columns_imp_feat:
            print(name[0], elem, sep = " ")
        print("-------")

number_imp_feat 2
s_cl 2
animate 2
gender 2
number 2
case1 2
reflection 2
perfective 2
transitive 2
person 2
tense 2
voice 2
degree 2
static 2
-------
number_imp_feat True
s_cl True
animate True
gender True
number True
case1 True
reflection True
perfective True
transitive True
person True
tense True
voice True
degree True
static True
-------
number_imp_feat False
s_cl False
animate False
gender False
number False
case1 False
reflection False
perfective False
transitive False
person False
tense False
voice False
degree False
static False
-------
number_imp_feat False
s_cl False
animate False
gender False
number False
case1 False
reflection False
perfective False
transitive False
person False
tense False
voice False
degree False
static False
-------
number_imp_feat False
s_cl False
animate False
gender False
number False
case1 False
reflection False
perfective False
transitive False
person False
tense False
voice False
degree False
static False
-------
number_imp_feat True
s_cl True
anim

In [47]:
def get_imp_feat_dep_3_model(word):
    imp_dep = db.query("WITH w AS (SELECT number_word FROM word WHERE word_text = \'" + word + "\'), mod AS (SELECT ref_to_dep_word,imp_feat_main, imp_feat_dep, prep, mark FROM model_3_level, w WHERE model_3_level.ref_to_main_word = w.number_word), impDep AS (SELECT important_features.* FROM mod, important_features WHERE mod.imp_feat_dep = important_features.number_imp_feat) SELECT * FROM impDep;")
    for str1 in imp_dep:
        for elem in str1:
            for name in columns_imp_feat:
                print(name[0], elem, sep = " ")
            print("-------")
    return imp_dep

In [49]:
e = get_imp_feat_dep_3_model('видеть')

number_imp_feat 2
s_cl 2
animate 2
gender 2
number 2
case1 2
reflection 2
perfective 2
transitive 2
person 2
tense 2
voice 2
degree 2
static 2
-------
number_imp_feat True
s_cl True
animate True
gender True
number True
case1 True
reflection True
perfective True
transitive True
person True
tense True
voice True
degree True
static True
-------
number_imp_feat False
s_cl False
animate False
gender False
number False
case1 False
reflection False
perfective False
transitive False
person False
tense False
voice False
degree False
static False
-------
number_imp_feat False
s_cl False
animate False
gender False
number False
case1 False
reflection False
perfective False
transitive False
person False
tense False
voice False
degree False
static False
-------
number_imp_feat False
s_cl False
animate False
gender False
number False
case1 False
reflection False
perfective False
transitive False
person False
tense False
voice False
degree False
static False
-------
number_imp_feat True
s_cl True
anim

In [50]:
e

[(2,
  True,
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False),
 (2,
  True,
  False,
  False,
  False,
  True,
  False,
  False,
  False,
  False,
  False,
  False,
  False,
  False)]

In [6]:
from Types import *

In [7]:
Es_cl

<enum 'Es_cl'>